In [29]:
import pandas as pd
import numpy as np
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt

from script import *
from feature_engineering import *
import db_operations as dbop
from constants import *
import data_process as dp

import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [30]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','std','size']

In [31]:
start = 20100101
cursor = dbop.connect_db("sqlite3").cursor()

df_d = dbop.create_df(cursor, STOCK_DAY[TABLE], start=start)
df_d = dp.proc_stock_d(dp.prepare_stock_d(df_d))
df_d.drop(columns=['open0','high0','low0','vol0'],inplace=True)
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
df_d['amt']/=1e5
df_d = df_d.astype('float32')
print(df_d.columns)
print(df_d.shape)
df_d.info(memory_usage='deep')
df_d.tail()

select * from stock_day where date>=20100101
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct'],
      dtype='object')
(6967224, 11)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6967224 entries, (000001.SZ, 2010-01-04 00:00:00) to (688399.SH, 2020-04-27 00:00:00)
Data columns (total 11 columns):
open          float32
high          float32
low           float32
close         float32
vol           float32
amt           float32
adj_factor    float32
avg           float32
close0        float32
avg0          float32
pct           float32
dtypes: float32(11)
memory usage: 639.4 MB


open        high         low       close  \
code      date                                                         
688399.SH 2020-04-21  108.000000  118.199997  102.000000  114.440002   
          2020-04-22  108.000000  118.500000  107.489998  109.089996   
          2020-04-23  117.029999  128.880005  117.029999  128.880005   
          2020-04-24  150.000000  153.000000  138.080002  144.080002   
          2020-04-27  135.000000  140.000000  126.000000  126.199997   

                               vol       amt  adj_factor         avg  \
code      date                                                         
688399.SH 2020-04-21  33081.828125  3.580782         1.0  108.240166   
          2020-04-22  21584.169922  2.432959         1.0  112.719612   
          2020-04-23  36237.828125  4.511718         1.0  124.502983   
          2020-04-24  46408.371094  6.756642         1.0  145.591034   
          2020-04-27  35730.238281  4.691092         1.0  131.291931   

                          close0        avg0        pct  
code      date                                           
688399.SH 2020-04-21  114.440002  108.240166   5.251539  
          2020-04-22  109.089996  112.719612  -4.674941  
          2020-04-23  128.880005  124.502983  18.140995  
          2020-04-24  144.080002  145.591034  11.793911  
          2020-04-27  126.199997  131.291931 -12.409777

In [32]:
df_d_basic = dbop.create_df(cursor, STOCK_DAILY_BASIC[TABLE], start=start)
df_d_basic = dp.prepare_stock_d_basic(df_d_basic)
#
df_d_basic["pb*pe_ttm"] = df_d_basic["pb"] * df_d_basic["pe_ttm"]
df_d_basic["pb*pe"] = df_d_basic["pb"] * df_d_basic["pe"]

df_d_basic = df_d_basic.astype('float32')
print(df_d_basic.columns)
print(df_d_basic.shape)
df_d_basic.info(memory_usage='deep')
df_d_basic.tail()

select * from stock_daily_basic where date>=20100101
Index(['close', 'turnover_rate', 'turnover_rate_f', 'volume_ratio', 'pe',
       'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share', 'float_share',
       'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm', 'pb*pe'],
      dtype='object')
(6588366, 16)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6588366 entries, (000001.SZ, 2010-01-04 00:00:00) to (688399.SH, 2020-04-27 00:00:00)
Data columns (total 16 columns):
close              float32
turnover_rate      float32
turnover_rate_f    float32
volume_ratio       float32
pe                 float32
pe_ttm             float32
pb                 float32
ps                 float32
ps_ttm             float32
total_share        float32
float_share        float32
free_share         float32
total_mv           float32
circ_mv            float32
pb*pe_ttm          float32
pb*pe              float32
dtypes: float32(16)
memory usage: 427.5 MB


close  turnover_rate  turnover_rate_f  \
code      date                                                     
688399.SH 2020-04-21  114.440002      24.801800        24.801800   
          2020-04-22  109.089996      16.181900        16.181900   
          2020-04-23  128.880005      27.167900        27.167900   
          2020-04-24  144.080002      34.792900        34.792900   
          2020-04-27  126.199997      26.787399        26.787399   

                      volume_ratio          pe     pe_ttm      pb         ps  \
code      date                                                                 
688399.SH 2020-04-21          1.54  105.108002  75.192001  6.5369  29.078699   
          2020-04-22          0.81  100.194298  71.676804  6.2313  27.719299   
          2020-04-23          1.25  118.370499  84.679703  7.3617  32.747898   
          2020-04-24          1.55  100.839996  48.565300  7.5117  29.245600   
          2020-04-27          1.04   88.325897  42.538399  6.5795  25.616301   

                         ps_ttm  total_share  float_share   free_share  \
code      date                                                           
688399.SH 2020-04-21  23.053400       5862.0  1333.847046  1333.847046   
          2020-04-22  21.975599       5862.0  1333.847046  1333.847046   
          2020-04-23  25.962200       5862.0  1333.847046  1333.847046   
          2020-04-24  19.375700       5862.0  1333.847046  1333.847046   
          2020-04-27  16.971201       5862.0  1333.847046  1333.847046   

                         total_mv        circ_mv   pb*pe_ttm       pb*pe  
code      date                                                            
688399.SH 2020-04-21  670847.2500  152645.453125  491.522583  687.080505  
          2020-04-22  639485.5625  145509.375000  446.639648  624.340698  
          2020-04-23  755494.5625  171906.203125  623.386597  871.408081  
          2020-04-24  844596.9375  192180.671875  364.807983  757.479797  
          2020-04-27  739784.3750  168331.484375  279.881409  581.140259

In [33]:
df = df_d.join(df_d_basic.drop(columns=['close']),how='left')
periods = [5,10,20,30,40,50,60,70,80,120,250]
for k in periods:
    df['{:d}ma'.format(k)]=df.reset_index('code').groupby('code')['close'].rolling(k).mean()
    df['{:d}ma_avg'.format(k)]=df.reset_index('code').groupby('code')['amt'].rolling(k).sum()\
            /df.reset_index('code').groupby('code')['vol'].rolling(k).sum()*1e6
print(df.columns)
    
for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]

for col2 in ["{}ma_avg".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]

print(df.columns)

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '5ma_avg', '10ma', '10ma_avg', '20ma',
       '20ma_avg', '30ma', '30ma_avg', '40ma', '40ma_avg', '50ma', '50ma_avg',
       '60ma', '60ma_avg', '70ma', '70ma_avg', '80ma', '80ma_avg', '120ma',
       '120ma_avg', '250ma', '250ma_avg'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '5ma_avg', '10ma', '10ma_avg', '20ma',
       '20ma_avg', '30ma', '30ma_avg', '40ma', '40ma_avg', '50ma', '50ma_avg',
       '60ma', '60ma_avg', '70ma', '70ma_avg', '80ma', '80ma_avg', '120ma',
       '120ma_avg', '250ma', '250ma_avg', 'close/5ma', 'close/10ma',
       'close/20ma', 'close/30ma', 'close/40ma', 'close/50ma', 'close/60ma',
       'close/70ma', 'close/80ma', 'close/120ma', 'close/250ma',
       'close/5ma_avg', 'close/10ma_avg', 'close/20ma_avg', 'close/30ma_avg',
       'close/40ma_avg', 'close/

In [34]:
start_date = '-'.join([str(start)[:4],str(start)[4:6],str(start)[6:8]])
df_r_spl = pd.read_parquet(r"database\return_spl_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()
df_r_spc = pd.read_parquet(r"database\return_spc_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()

features = []
for col2 in ["{}ma".format(k) for k in periods]+["{}ma_avg".format(k) for k in periods]:
    col1 = 'close'
    features.append('{}/{}'.format(col1,col2))
print(features)

C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:707: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:734: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\dell-pc\Anaconda3.6\lib\site-packages\pyarrow\pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


['close/5ma', 'close/10ma', 'close/20ma', 'close/30ma', 'close/40ma', 'close/50ma', 'close/60ma', 'close/70ma', 'close/80ma', 'close/120ma', 'close/250ma', 'close/5ma_avg', 'close/10ma_avg', 'close/20ma_avg', 'close/30ma_avg', 'close/40ma_avg', 'close/50ma_avg', 'close/60ma_avg', 'close/70ma_avg', 'close/80ma_avg', 'close/120ma_avg', 'close/250ma_avg']


In [8]:
# result_50 = ml.assess_feature3(df[features],df_r.r,q_bin=50)
stats_1000_spl,result_1000_spl = ml.assess_feature3(df[features],df_r_spl.r,q_bin=1000)
stats_1000_spc,result_1000_spc = ml.assess_feature3(df[features],df_r_spc.r,q_bin=1000)

# print(result_50.sort_values(['median_std','mean_std'],ascending=False))
stats_1000_spl.sort_values(['median_std','mean_std'],ascending=False)
stats_1000_spc.sort_values(['median_std','mean_std'],ascending=False)

stats_100_spl,result_100_spl = ml.assess_feature3(df[features],df_r_spl.r,q_bin=100)
stats_100_spc,result_100_spc = ml.assess_feature3(df[features],df_r_spc.r,q_bin=100)

# print(result_50.sort_values(['median_std','mean_std'],ascending=False))
stats_100_spl.sort_values(['median_std','mean_std'],ascending=False)
stats_100_spc.sort_values(['median_std','mean_std'],ascending=False)

close/10ma 
                                    mean    median
bin                                              
(0.5809000000000001, 0.73171]  0.062235  0.076305
(0.73171, 0.76828]             0.002703  0.010144
(0.76828, 0.79142]            -0.026530 -0.014742
(0.79142, 0.80939]            -0.013303 -0.011884
(0.80939, 0.82264]            -0.009872 -0.009934
...                                 ...       ...
(1.19023, 1.20441]            -0.007542 -0.017544
(1.20441, 1.22495]            -0.011367 -0.021983
(1.22495, 1.25804]            -0.013546 -0.023355
(1.25804, 1.34014]            -0.015138 -0.028985
(1.34014, 5.52254]            -0.014768 -0.029143

[1000 rows x 2 columns] 

close/10ma_avg 
                         mean    median
bin                                   
(0.41812, 0.71981]  0.070616  0.081017
(0.71981, 0.75655] -0.006242  0.000000
(0.75655, 0.78122] -0.013058 -0.011662
(0.78122, 0.79887] -0.017487 -0.014004
(0.79887, 0.81227] -0.015890 -0.011913
...                 

close/5ma_avg 
                                    mean    median
bin                                              
(0.6025900000000001, 0.80966]  0.062599  0.052119
(0.80966, 0.8361]             -0.012832 -0.001019
(0.8361, 0.85105]             -0.001219  0.001577
(0.85105, 0.86186]            -0.006341 -0.005287
(0.86186, 0.87022]            -0.001396 -0.001326
...                                 ...       ...
(1.11513, 1.12336]            -0.013196 -0.019920
(1.12336, 1.13478]            -0.012338 -0.020400
(1.13478, 1.15138]            -0.016056 -0.022486
(1.15138, 1.17929]            -0.027866 -0.036086
(1.17929, 1.57965]            -0.038142 -0.052649

[1000 rows x 2 columns] 

close/60ma 
                       mean    median
bin                                 
(0.3089, 0.6092]  0.105373  0.086539
(0.6092, 0.6471]  0.038684  0.041040
(0.6471, 0.6695]  0.025224  0.032187
(0.6695, 0.6849]  0.021604  0.024451
(0.6849, 0.697]   0.018935  0.020510
...                    ...       ..

close/30ma 
                       mean    median
bin                                 
(0.3689, 0.6137]  0.108950  0.152248
(0.6137, 0.6705]  0.013926  0.009245
(0.6705, 0.7001]  0.021105  0.013431
(0.7001, 0.7198]  0.016880  0.006426
(0.7198, 0.7344]  0.015455  0.002856
...                    ...       ...
(1.3363, 1.3579] -0.010997 -0.033306
(1.3579, 1.3858] -0.012888 -0.034230
(1.3858, 1.4269] -0.012610 -0.035874
(1.4269, 1.5046] -0.016902 -0.037551
(1.5046, 8.3046] -0.020985 -0.044573

[1000 rows x 2 columns] 

close/30ma_avg 
                       mean    median
bin                                 
(0.3539, 0.6081]  0.111094  0.159753
(0.6081, 0.6609]  0.020569  0.024706
(0.6609, 0.6877]  0.026319  0.023309
(0.6877, 0.7057]  0.021531  0.010537
(0.7057, 0.7191]  0.015012  0.001616
...                    ...       ...
(1.2757, 1.2934] -0.007687 -0.031343
(1.2934, 1.3165] -0.007212 -0.029969
(1.3165, 1.3501] -0.010482 -0.032800
(1.3501, 1.4119] -0.012658 -0.036110
(1.4119, 2.2896] -

,mean_std,median_std,mean_max,median_max,mean_min,median_min,mean_q96%,median_q96%,mean_q4%,median_q4%
close/40ma_avg,0.004120,0.005703,0.107964,0.123377,-0.013047,-0.024571,0.005915,-0.003541,-0.001153,-0.012271
close/30ma_avg,0.003563,0.005699,0.093333,0.124930,-0.016573,-0.027534,0.004723,-0.004564,-0.001171,-0.012556
close/40ma,0.004155,0.005547,0.106528,0.114582,-0.016800,-0.028801,0.006597,-0.002983,-0.001035,-0.012236
close/50ma_avg,0.004428,0.005463,0.111969,0.110239,-0.010806,-0.021808,0.006650,-0.003036,-0.001113,-0.012396
close/30ma,0.003537,0.005453,0.092001,0.115826,-0.018345,-0.031024,0.005010,-0.003821,-0.001058,-0.012506
close/50ma,0.004342,0.005264,0.107192,0.097460,-0.015674,-0.027257,0.007101,-0.002621,-0.001004,-0.012252
close/60ma_avg,0.004551,0.005222,0.113000,0.097268,-0.012114,-0.022529,0.006797,-0.002560,-0.000923,-0.012282
close/60ma,0.004382,0.005093,0.105373,0.086539,-0.014155,-0.026166,0.007212,-0.002262,-0.000942,-0.012282
close/70ma_avg,0.004452,0.004962,0.100893,0.081984,-0.011670,-0.019730,0.008164,-0.001834,-0.001130,-0.012225
close/20ma_avg,0.003006,0.004937,0.071685,0.107302,-0.020463,-0.032529,0.005034,-0.004687,-0.001012,-0.012912


,mean_std,median_std,mean_max,median_max,mean_min,median_min,mean_q96%,median_q96%,mean_q4%,median_q4%
close/50ma_avg,0.006011,0.007524,0.134488,0.161755,-0.012909,-0.036735,0.010617,-0.008688,-0.001322,-0.022877
close/40ma_avg,0.005431,0.007341,0.127399,0.163793,-0.015259,-0.038153,0.009826,-0.009517,-0.001335,-0.022713
close/50ma,0.005935,0.007300,0.124400,0.146303,-0.017967,-0.042426,0.011530,-0.007673,-0.001463,-0.023431
close/40ma,0.005579,0.007292,0.123858,0.156245,-0.019312,-0.043831,0.010981,-0.008252,-0.001792,-0.023105
close/60ma_avg,0.006021,0.007043,0.127486,0.137874,-0.014337,-0.037433,0.011772,-0.007777,-0.001313,-0.023382
close/30ma_avg,0.004530,0.006821,0.111094,0.159753,-0.019754,-0.042232,0.007467,-0.011471,-0.001549,-0.023043
close/30ma,0.004664,0.006657,0.108950,0.152248,-0.020985,-0.044573,0.009052,-0.010167,-0.001740,-0.022933
close/60ma,0.005843,0.006491,0.116188,0.101593,-0.015617,-0.040194,0.012053,-0.006915,-0.001617,-0.023752
close/70ma_avg,0.005915,0.006024,0.107307,0.077849,-0.015344,-0.035608,0.013190,-0.006334,-0.001481,-0.023561
close/70ma,0.005781,0.005968,0.101710,0.062098,-0.014462,-0.038462,0.013758,-0.005940,-0.001958,-0.024121


...

close/5ma_avg 
                       mean    median
bin                                 
(0.6025, 0.8943]  0.003986 -0.015933
(0.8943, 0.9153]  0.003295 -0.015251
(0.9153, 0.9269]  0.005968 -0.013889
(0.9269, 0.9349]  0.007440 -0.013034
(0.9349, 0.941]   0.007683 -0.013224
(0.941, 0.9458]   0.007881 -0.012739
(0.9458, 0.9498]  0.006849 -0.013245
(0.9498, 0.9532]  0.006982 -0.013468
(0.9532, 0.9562]  0.006585 -0.013740
(0.9562, 0.9587]  0.005007 -0.014887
(0.9587, 0.9611]  0.005528 -0.013986
(0.9611, 0.9632]  0.005229 -0.014368
(0.9632, 0.9651]  0.004917 -0.014992
(0.9651, 0.9669]  0.004177 -0.015424
(0.9669, 0.9685]  0.003370 -0.016013
(0.9685, 0.9701]  0.003733 -0.015642
(0.9701, 0.9715]  0.003209 -0.016260
(0.9715, 0.9729]  0.002327 -0.016567
(0.9729, 0.9742]  0.002242 -0.016636
(0.9742, 0.9754]  0.001430 -0.016878
(0.9754, 0.9766]  0.001170 -0.017196
(0.9766, 0.9777]  0.001625 -0.017065
(0.9777, 0.9788]  0.001497 -0.016611
(0.9788, 0.9798]  0.001012 -0.017046
(0.9798, 0.9808]  0.00

close/70ma 
                     mean    median
bin                               
(0.33, 0.723]   0.037687  0.014350
(0.723, 0.763]  0.019373 -0.001271
(0.763, 0.787]  0.015587 -0.004079
(0.787, 0.804]  0.014351 -0.005882
(0.804, 0.818]  0.012467 -0.007146
(0.818, 0.83]   0.012018 -0.006363
(0.83, 0.84]    0.011405 -0.006911
(0.84, 0.848]   0.010503 -0.007634
(0.848, 0.856]  0.009660 -0.008097
(0.856, 0.863]  0.008541 -0.009273
(0.863, 0.869]  0.008347 -0.008881
(0.869, 0.875]  0.007920 -0.009238
(0.875, 0.88]   0.006743 -0.009669
(0.88, 0.886]   0.007061 -0.009302
(0.886, 0.89]   0.006265 -0.009788
(0.89, 0.895]   0.005821 -0.010638
(0.895, 0.899]  0.004801 -0.011182
(0.899, 0.903]  0.004955 -0.010756
(0.903, 0.907]  0.004710 -0.011369
(0.907, 0.911]  0.004315 -0.011547
(0.911, 0.914]  0.003902 -0.011999
(0.914, 0.918]  0.003174 -0.012543
(0.918, 0.921]  0.003566 -0.012179
(0.921, 0.924]  0.003390 -0.012146
(0.924, 0.927]  0.002692 -0.012987
(0.927, 0.93]   0.002938 -0.012500
(0.93, 

close/80ma_avg 
                     mean    median
bin                               
(0.361, 0.691]  0.040533  0.016610
(0.691, 0.73]   0.023551  0.002025
(0.73, 0.754]   0.018369 -0.002110
(0.754, 0.771]  0.014782 -0.005264
(0.771, 0.785]  0.011624 -0.007914
(0.785, 0.797]  0.010636 -0.008345
(0.797, 0.807]  0.010024 -0.007732
(0.807, 0.816]  0.009352 -0.008417
(0.816, 0.824]  0.008802 -0.009077
(0.824, 0.832]  0.007492 -0.010603
(0.832, 0.838]  0.006468 -0.010463
(0.838, 0.845]  0.007001 -0.010092
(0.845, 0.851]  0.005298 -0.011317
(0.851, 0.856]  0.005061 -0.011159
(0.856, 0.861]  0.004912 -0.011407
(0.861, 0.866]  0.005156 -0.010677
(0.866, 0.871]  0.005344 -0.011203
(0.871, 0.875]  0.004642 -0.011392
(0.875, 0.88]   0.004526 -0.011334
(0.88, 0.884]   0.003311 -0.012058
(0.884, 0.888]  0.003486 -0.011561
(0.888, 0.892]  0.003692 -0.012172
(0.892, 0.895]  0.003380 -0.012034
(0.895, 0.899]  0.002984 -0.012484
(0.899, 0.902]  0.002387 -0.013343
(0.902, 0.906]  0.002265 -0.013058
(0.

,mean_std,median_std,mean_max,median_max,mean_min,median_min,mean_q96%,median_q96%,mean_q4%,median_q4%
close/70ma_avg,0.003503,0.004223,0.028497,0.022556,-0.004605,-0.013652,0.007466,-0.002808,-0.000052,-0.011426
close/80ma,0.003593,0.004204,0.028570,0.021429,-0.005557,-0.015455,0.008183,-0.001892,-0.000806,-0.011263
close/70ma,0.003383,0.004189,0.027237,0.021956,-0.005590,-0.015625,0.007466,-0.002146,-0.000714,-0.011111
close/60ma_avg,0.003341,0.004178,0.028345,0.023426,-0.004436,-0.013699,0.006645,-0.002621,-0.000056,-0.011595
close/80ma_avg,0.003655,0.004178,0.029495,0.021860,-0.005230,-0.014273,0.007100,-0.003166,-0.000322,-0.011457
close/60ma,0.003252,0.004134,0.027107,0.022132,-0.005620,-0.015748,0.006507,-0.003208,-0.000105,-0.011669
close/50ma,0.003187,0.004105,0.026578,0.022033,-0.005757,-0.016097,0.006440,-0.003666,-0.000196,-0.011784
close/50ma_avg,0.003186,0.004096,0.027400,0.023030,-0.003827,-0.013253,0.005836,-0.003300,-0.000239,-0.011801
close/40ma,0.002869,0.004001,0.022791,0.021429,-0.006738,-0.017471,0.005811,-0.003693,-0.000204,-0.011573
close/40ma_avg,0.002799,0.003932,0.023517,0.022495,-0.004471,-0.014017,0.005393,-0.004334,-0.000177,-0.011696


,mean_std,median_std,mean_max,median_max,mean_min,median_min,mean_q96%,median_q96%,mean_q4%,median_q4%
close/80ma,0.005462,0.005607,0.039434,0.014458,-0.008217,-0.031798,0.013525,-0.006008,-0.001848,-0.023956
close/70ma,0.005187,0.005526,0.037687,0.014350,-0.008381,-0.031633,0.012542,-0.006344,-0.001627,-0.024000
close/80ma_avg,0.005470,0.005386,0.040533,0.016610,-0.007707,-0.030674,0.011750,-0.007633,-0.000945,-0.022599
close/120ma,0.005491,0.005360,0.038009,0.010000,-0.007065,-0.030982,0.014178,-0.005118,-0.001101,-0.023380
close/60ma,0.004991,0.005357,0.038278,0.015000,-0.008289,-0.031427,0.011080,-0.007890,-0.001104,-0.023233
close/70ma_avg,0.005259,0.005353,0.039044,0.016013,-0.006851,-0.030086,0.012511,-0.007428,-0.000757,-0.022686
close/60ma_avg,0.005028,0.005240,0.039884,0.017369,-0.006535,-0.030159,0.011563,-0.008269,-0.000652,-0.022778
close/50ma,0.004946,0.005182,0.038696,0.015707,-0.008627,-0.031431,0.011156,-0.008279,-0.000912,-0.023122
close/120ma_avg,0.005555,0.005170,0.042124,0.015464,-0.005271,-0.029258,0.014467,-0.003921,0.000483,-0.021917
close/50ma_avg,0.004836,0.004993,0.039801,0.017371,-0.005851,-0.029496,0.010273,-0.009407,-0.000349,-0.022533


In [10]:
stats_100_spl.sort_values('mean_max',ascending=False)
stats_100_spc.sort_values('mean_max',ascending=False)

stats_1000_spl.sort_values('mean_max',ascending=False)
stats_1000_spc.sort_values('mean_max',ascending=False)


,mean_std,median_std,mean_max,median_max,mean_min,median_min,mean_q96%,median_q96%,mean_q4%,median_q4%
close/120ma_avg,0.003725,0.003841,0.030206,0.018634,-0.003394,-0.012500,0.008916,-0.000972,0.000349,-0.010814
close/80ma_avg,0.003655,0.004178,0.029495,0.021860,-0.005230,-0.014273,0.007100,-0.003166,-0.000322,-0.011457
close/80ma,0.003593,0.004204,0.028570,0.021429,-0.005557,-0.015455,0.008183,-0.001892,-0.000806,-0.011263
close/70ma_avg,0.003503,0.004223,0.028497,0.022556,-0.004605,-0.013652,0.007466,-0.002808,-0.000052,-0.011426
close/60ma_avg,0.003341,0.004178,0.028345,0.023426,-0.004436,-0.013699,0.006645,-0.002621,-0.000056,-0.011595
close/50ma_avg,0.003186,0.004096,0.027400,0.023030,-0.003827,-0.013253,0.005836,-0.003300,-0.000239,-0.011801
close/70ma,0.003383,0.004189,0.027237,0.021956,-0.005590,-0.015625,0.007466,-0.002146,-0.000714,-0.011111
close/120ma,0.003623,0.003738,0.027160,0.015695,-0.004895,-0.014724,0.009075,-0.001703,-0.000251,-0.011094
close/60ma,0.003252,0.004134,0.027107,0.022132,-0.005620,-0.015748,0.006507,-0.003208,-0.000105,-0.011669
close/50ma,0.003187,0.004105,0.026578,0.022033,-0.005757,-0.016097,0.006440,-0.003666,-0.000196,-0.011784


,mean_std,median_std,mean_max,median_max,mean_min,median_min,mean_q96%,median_q96%,mean_q4%,median_q4%
close/120ma_avg,0.005555,0.005170,0.042124,0.015464,-0.005271,-0.029258,0.014467,-0.003921,0.000483,-0.021917
close/80ma_avg,0.005470,0.005386,0.040533,0.016610,-0.007707,-0.030674,0.011750,-0.007633,-0.000945,-0.022599
close/60ma_avg,0.005028,0.005240,0.039884,0.017369,-0.006535,-0.030159,0.011563,-0.008269,-0.000652,-0.022778
close/50ma_avg,0.004836,0.004993,0.039801,0.017371,-0.005851,-0.029496,0.010273,-0.009407,-0.000349,-0.022533
close/80ma,0.005462,0.005607,0.039434,0.014458,-0.008217,-0.031798,0.013525,-0.006008,-0.001848,-0.023956
close/70ma_avg,0.005259,0.005353,0.039044,0.016013,-0.006851,-0.030086,0.012511,-0.007428,-0.000757,-0.022686
close/50ma,0.004946,0.005182,0.038696,0.015707,-0.008627,-0.031431,0.011156,-0.008279,-0.000912,-0.023122
close/60ma,0.004991,0.005357,0.038278,0.015000,-0.008289,-0.031427,0.011080,-0.007890,-0.001104,-0.023233
close/120ma,0.005491,0.005360,0.038009,0.010000,-0.007065,-0.030982,0.014178,-0.005118,-0.001101,-0.023380
close/70ma,0.005187,0.005526,0.037687,0.014350,-0.008381,-0.031633,0.012542,-0.006344,-0.001627,-0.024000


,mean_std,median_std,mean_max,median_max,mean_min,median_min,mean_q96%,median_q96%,mean_q4%,median_q4%
close/60ma_avg,0.004551,0.005222,0.113000,0.097268,-0.012114,-0.022529,0.006797,-0.002560,-0.000923,-0.012282
close/50ma_avg,0.004428,0.005463,0.111969,0.110239,-0.010806,-0.021808,0.006650,-0.003036,-0.001113,-0.012396
close/40ma_avg,0.004120,0.005703,0.107964,0.123377,-0.013047,-0.024571,0.005915,-0.003541,-0.001153,-0.012271
close/50ma,0.004342,0.005264,0.107192,0.097460,-0.015674,-0.027257,0.007101,-0.002621,-0.001004,-0.012252
close/40ma,0.004155,0.005547,0.106528,0.114582,-0.016800,-0.028801,0.006597,-0.002983,-0.001035,-0.012236
close/60ma,0.004382,0.005093,0.105373,0.086539,-0.014155,-0.026166,0.007212,-0.002262,-0.000942,-0.012282
close/70ma_avg,0.004452,0.004962,0.100893,0.081984,-0.011670,-0.019730,0.008164,-0.001834,-0.001130,-0.012225
close/70ma,0.004312,0.004890,0.097087,0.076276,-0.012887,-0.024808,0.008045,-0.001280,-0.001042,-0.012031
close/30ma_avg,0.003563,0.005699,0.093333,0.124930,-0.016573,-0.027534,0.004723,-0.004564,-0.001171,-0.012556
close/30ma,0.003537,0.005453,0.092001,0.115826,-0.018345,-0.031024,0.005010,-0.003821,-0.001058,-0.012506


,mean_std,median_std,mean_max,median_max,mean_min,median_min,mean_q96%,median_q96%,mean_q4%,median_q4%
close/50ma_avg,0.006011,0.007524,0.134488,0.161755,-0.012909,-0.036735,0.010617,-0.008688,-0.001322,-0.022877
close/60ma_avg,0.006021,0.007043,0.127486,0.137874,-0.014337,-0.037433,0.011772,-0.007777,-0.001313,-0.023382
close/40ma_avg,0.005431,0.007341,0.127399,0.163793,-0.015259,-0.038153,0.009826,-0.009517,-0.001335,-0.022713
close/50ma,0.005935,0.007300,0.124400,0.146303,-0.017967,-0.042426,0.011530,-0.007673,-0.001463,-0.023431
close/40ma,0.005579,0.007292,0.123858,0.156245,-0.019312,-0.043831,0.010981,-0.008252,-0.001792,-0.023105
close/60ma,0.005843,0.006491,0.116188,0.101593,-0.015617,-0.040194,0.012053,-0.006915,-0.001617,-0.023752
close/30ma_avg,0.004530,0.006821,0.111094,0.159753,-0.019754,-0.042232,0.007467,-0.011471,-0.001549,-0.023043
close/30ma,0.004664,0.006657,0.108950,0.152248,-0.020985,-0.044573,0.009052,-0.010167,-0.001740,-0.022933
close/70ma_avg,0.005915,0.006024,0.107307,0.077849,-0.015344,-0.035608,0.013190,-0.006334,-0.001481,-0.023561
close/70ma,0.005781,0.005968,0.101710,0.062098,-0.014462,-0.038462,0.013758,-0.005940,-0.001958,-0.024121


In [36]:
df.columns
# df = df.join(df_d_basic.drop(columns=['close']),how='left')
index = df_d.index[(df_d['close/120ma']<=0.69)
                 & (df_d['close/30ma']<=0.778)
                 & (df_d['close/60ma']<=0.65)
                 & (df_d['close/60ma']>=0.55)
                ]
len(index)
# print(df['turnover_rate_f'].head())
# df['p10mean_turnover_rate_f'] = df.reset_index('code').groupby('code')['turnover_rate_f'].rolling(10).mean()
df['p10mean_turnover_rate'] = df.reset_index('code').groupby('code')['turnover_rate'].rolling(10).mean()

stock_mask = (df['circ_mv']>50e4) & (df['total_mv']>100e4) & (df['close0']>2.5) & (df['p10mean_turnover_rate']<5) & (df['amt']>0.5)
print(stock_mask.sum(),stock_mask.mean())
# print(len(index))

strategy_mask = (
    (df['close/120ma']<0.72) 
    & (df['close/60ma']<0.77) 
    & (df['close/60ma']>0.55) 
    & (df['close/30ma']<0.82) 
#     & (df['close/20ma']<0.82)
#     & (df['close/5ma']<0.95)
                )
print(strategy_mask.sum())

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '5ma_avg', '10ma', '10ma_avg', '20ma',
       '20ma_avg', '30ma', '30ma_avg', '40ma', '40ma_avg', '50ma', '50ma_avg',
       '60ma', '60ma_avg', '70ma', '70ma_avg', '80ma', '80ma_avg', '120ma',
       '120ma_avg', '250ma', '250ma_avg', 'close/5ma', 'close/10ma',
       'close/20ma', 'close/30ma', 'close/40ma', 'close/50ma', 'close/60ma',
       'close/70ma', 'close/80ma', 'close/120ma', 'close/250ma',
       'close/5ma_avg', 'close/10ma_avg', 'close/20ma_avg', 'close/30ma_avg',
       'close/40ma_avg', 'close/50ma_avg', 'close/60ma_avg', 'close/70ma_avg',
       'close/80ma_avg', 'close/120ma_avg', 'close/250ma_avg'],
      dtype='object')

3711

1213960 0.17423869248354867
21872


In [37]:
df_r_spl.loc[df.index[stock_mask & strategy_mask],'r'].agg(agg_operations)
df_r_spc.loc[df.index[stock_mask & strategy_mask],'r'].agg(agg_operations)
# print(df_r_spl2.loc[df.index[stock_mask & strategy_mask],'r'].agg(agg_operations))
df_r_spc.loc[df.index[stock_mask & strategy_mask],'r'].reset_index('code')['r'].resample('Q').agg(agg_operations)
df_r_spc.loc[df.index[stock_mask & strategy_mask],:].sort_values('date',ascending=False)[:50]

df_r_spl.loc[index,'r'].agg(agg_operations)
df_r_spc.loc[index,'r'].agg(agg_operations)
df_r_spc.loc[index,:].reset_index('code')['r'].resample('Q').agg(agg_operations)
df_r_spc.loc[index,:].sort_values('date',ascending=False)[:50]


mean         0.028329
median       0.024216
max          0.944298
min         -0.554711
std          0.087770
size      3205.000000
Name: r, dtype: float64

mean         0.034210
median       0.016949
max          0.944298
min         -0.554711
std          0.106377
size      3205.000000
Name: r, dtype: float64

,mean,median,max,min,std,size
date,,,,,,
2011-12-31,-0.019393,-0.025522,0.153846,-0.103537,0.056503,24
2012-03-31,0.137612,0.145854,0.246291,0.040000,0.066942,7
2012-06-30,NaN,NaN,NaN,NaN,NaN,0
2012-09-30,NaN,NaN,NaN,NaN,NaN,0
2012-12-31,0.179314,0.175102,0.203595,0.149301,0.021886,5
2013-03-31,NaN,NaN,NaN,NaN,NaN,0
2013-06-30,-0.025619,-0.027530,0.074227,-0.082334,0.031233,60
2013-09-30,-0.003226,-0.009333,0.087558,-0.071948,0.044924,37
2013-12-31,-0.009542,-0.036657,0.074426,-0.065434,0.062424,9


buy_at         max  max_idx   idx  is_selled  \
code      date                                                           
002506.SZ 2020-04-27         NaN         NaN      NaN  1885      False   
          2020-04-24   13.354139   13.761901   1885.0  1884      False   
          2020-04-23   13.608991   13.863840   1884.0  1883      False   
          2020-04-22   14.067720   14.118690   1883.0  1882       True   
          2020-04-21   14.067720   14.118690   1882.0  1881       True   
          2020-04-20   14.118690   14.628389   1881.0  1880       True   
300207.SZ 2020-04-13   91.053711  102.341362   2132.0  2130       True   
002411.SZ 2020-04-10   35.994240   38.868778   2095.0  2092       True   
002583.SZ 2020-04-03   33.788223   38.229603   2107.0  2100       True   
002411.SZ 2020-04-03   37.077396   38.910439   2089.0  2088       True   
002583.SZ 2020-04-02   34.013103   38.229603   2107.0  2099       True   
002411.SZ 2020-04-02   32.994720   38.910439   2089.0  2087       True   
002123.SZ 2020-04-01   97.448639  146.759995   2357.0  2351       True   
300618.SZ 2020-04-01  109.310043  121.387108    752.0   748       True   
002583.SZ 2020-04-01   32.832481   38.229603   2107.0  2098       True   
002411.SZ 2020-04-01   32.161518   38.910439   2089.0  2086       True   
601808.SH 2020-04-01   12.939040   15.308160   2488.0  2485       True   
002353.SZ 2020-04-01  183.529388  214.239792   2352.0  2346       True   
          2020-03-31  178.804703  214.239792   2352.0  2345       True   
300618.SZ 2020-03-31  106.319305  121.387108    752.0   747       True   
300766.SZ 2020-03-31   26.150000   31.400000    254.0   248       True   
002411.SZ 2020-03-31   32.328159   38.910439   2089.0  2085       True   
002583.SZ 2020-03-31   32.495163   38.229603   2107.0  2097       True   
002123.SZ 2020-03-31   97.448639  146.759995   2357.0  2350       True   
002353.SZ 2020-03-30  178.397400  207.722992   2348.0  2344       True   
601808.SH 2020-03-30   12.802361   15.308160   2488.0  2483       True   
300766.SZ 2020-03-30   26.000000   31.400000    254.0   247       True   
002411.SZ 2020-03-30   34.452820   34.452820   2085.0  2084       True   
300618.SZ 2020-03-30  106.547607  121.387108    752.0   746       True   
002583.SZ 2020-03-30   34.181763   38.229603   2107.0  2096       True   
002123.SZ 2020-03-30  101.484535  146.759995   2357.0  2349       True   
002353.SZ 2020-03-27  178.071564  207.722992   2348.0  2343       True   
300618.SZ 2020-03-27  106.091011  121.387108    752.0   745       True   
002411.SZ 2020-03-27   34.577801   35.994240   2084.0  2083       True   
002506.SZ 2020-03-26   17.839500   18.094351   1865.0  1864       True   
300618.SZ 2020-03-26  112.985672  113.236794    745.0   744       True   
002411.SZ 2020-03-26   33.744602   36.494160   2083.0  2082       True   
002353.SZ 2020-03-26  192.734360  193.630417   2343.0  2342       True   
002411.SZ 2020-03-25   38.285538   39.493675   2082.0  2081       True   
002506.SZ 2020-03-25   18.145321   20.184120   1864.0  1863       True   
002353.SZ 2020-03-25  192.734360  194.445023   2342.0  2341       True   
300618.SZ 2020-03-24  111.821335  115.451309    743.0   742       True   
002353.SZ 2020-03-24  192.978745  200.391602   2341.0  2340       True   
002411.SZ 2020-03-24   40.243557   41.118420   2081.0  2080       True   
300618.SZ 2020-03-23  108.807777  115.451309    743.0   741       True   
002387.SZ 2020-03-23   39.458221   42.553680   2269.0  2265       True   
002411.SZ 2020-03-23   40.951778   41.160080   2080.0  2079       True   
002008.SZ 2020-03-23  356.953125  378.648529   2459.0  2457       True   
300748.SZ 2020-03-23   27.178278   31.123999    364.0   360       True   
002563.SZ 2020-03-23   32.643448   37.198349   2190.0  2179       True   

                         sell_at  sell_date         r  
code      date                                         
002506.SZ 2020-04-27         NaN        NaT       NaN  
         

mean         0.058360
median       0.054930
max          1.204987
min         -0.735319
std          0.118516
size      3711.000000
Name: r, dtype: float64

mean         0.059863
median       0.035117
max          1.204987
min         -0.735319
std          0.138255
size      3711.000000
Name: r, dtype: float64

,mean,median,max,min,std,size
date,,,,,,
2011-09-30,0.018602,0.018602,0.018602,0.018602,NaN,1
2011-12-31,-0.019784,-0.038702,0.216216,-0.157175,0.089184,53
2012-03-31,0.092554,0.119785,0.436782,-0.099422,0.106455,59
2012-06-30,NaN,NaN,NaN,NaN,NaN,0
2012-09-30,NaN,NaN,NaN,NaN,NaN,0
2012-12-31,0.084579,0.138958,0.178434,-0.104598,0.104539,9
2013-03-31,NaN,NaN,NaN,NaN,NaN,0
2013-06-30,NaN,NaN,NaN,NaN,NaN,0
2013-09-30,NaN,NaN,NaN,NaN,NaN,0


buy_at         max  max_idx   idx  is_selled  \
code      date                                                           
002506.SZ 2020-04-27         NaN         NaN      NaN  1885      False   
002711.SZ 2020-04-27         NaN         NaN      NaN  1398      False   
          2020-04-24    5.390250    5.893340   1398.0  1397      False   
002506.SZ 2020-04-24   13.354139   13.761901   1885.0  1884      False   
          2020-04-23   13.608991   13.863840   1884.0  1883      False   
          2020-04-22   14.067720   14.118690   1883.0  1882       True   
          2020-04-21   14.067720   14.118690   1882.0  1881       True   
          2020-04-20   14.118690   14.628389   1881.0  1880       True   
          2020-04-17   14.322570   14.628389   1881.0  1879       True   
          2020-04-16   14.628389   15.087121   1879.0  1878       True   
          2020-04-15   14.526450   15.087121   1879.0  1877       True   
          2020-04-14   15.138090   15.392941   1877.0  1876       True   
          2020-04-10   14.322570   15.392941   1877.0  1874       True   
          2020-04-09   15.392941   15.392941   1874.0  1873       True   
          2020-04-08   15.494880   15.494880   1873.0  1872       True   
          2020-04-07   15.545850   15.596820   1872.0  1871       True   
          2020-04-03   15.494880   15.749730   1871.0  1870       True   
          2020-04-02   15.494880   15.851670   1870.0  1869       True   
002411.SZ 2020-04-02   32.994720   38.910439   2089.0  2087       True   
002506.SZ 2020-04-01   15.545850   15.851670   1870.0  1868       True   
002411.SZ 2020-04-01   32.161518   38.910439   2089.0  2086       True   
300618.SZ 2020-03-31  106.319305  121.387108    752.0   747       True   
002353.SZ 2020-03-31  178.804703  214.239792   2352.0  2345       True   
002411.SZ 2020-03-31   32.328159   38.910439   2089.0  2085       True   
002506.SZ 2020-03-31   15.392941   16.157492   1868.0  1867       True   
300618.SZ 2020-03-30  106.547607  121.387108    752.0   746       True   
002506.SZ 2020-03-30   16.412340   16.718161   1867.0  1866       True   
002353.SZ 2020-03-30  178.397400  207.722992   2348.0  2344       True   
002411.SZ 2020-03-30   34.452820   34.452820   2085.0  2084       True   
002506.SZ 2020-03-27   16.973009   17.482710   1866.0  1865       True   
002353.SZ 2020-03-27  178.071564  207.722992   2348.0  2343       True   
002411.SZ 2020-03-27   34.577801   35.994240   2084.0  2083       True   
          2020-03-26   33.744602   36.494160   2083.0  2082       True   
300618.SZ 2020-03-23  108.807777  115.451309    743.0   741       True   
600083.SH 2020-03-13   11.770000   12.110000   2340.0  2339       True   
          2020-03-12   11.310000   12.110000   2340.0  2338       True   
          2020-03-11   12.350000   12.360000   2338.0  2337       True   
          2020-03-10   12.990000   13.100000   2337.0  2336       True   
          2020-03-09   12.490000   13.100000   2337.0  2335       True   
          2020-03-06   13.300000   13.380000   2335.0  2334       True   
          2020-03-05   13.550000   13.970000   2334.0  2333       True   
          2020-03-04   13.670000   14.000000   2333.0  2332       True   
          2020-03-03   13.790000   14.000000   2333.0  2331       True   
          2020-03-02   13.440000   14.160000   2331.0  2330       True   
          2020-02-28   13.000000   14.160000   2331.0  2329       True   
600892.SH 2020-02-06   22.130501   26.177219   2285.0  2283       True   
          2020-02-05   20.612980   23.774479   2284.0  2282       True   
603012.SH 2020-02-04   13.858401   16.242861   1194.0  1182       True   
600069.SH 2020-02-04   10.777200   12.830000   2320.0  2317       True   
000662.SZ 2020-02-04   20.908781   24.413721   2099.0  2095       True   

                         sell_at  sell_date         r  
code      date                                         
002506.SZ 2020-04-27         NaN        NaT       NaN  
002711.SZ

In [38]:
index2 = df_d.index[(df_d['close/120ma_avg']<=0.64)
                 & (df_d['close/80ma_avg']<=0.68)
                 & (df_d['close/60ma_avg']<=0.7)
                ]
len(index2)

df_r_spl.loc[index2,'r'].agg(agg_operations)
df_r_spc.loc[index2,'r'].agg(agg_operations)
df_r_spc.loc[index2,:].reset_index('code')['r'].resample('Q').agg(agg_operations)
df_r_spc.loc[index2,:].sort_values('date',ascending=False)[:50].sort_index(ascending=False)

# strategy_mask2 = (
#     (df['close/120ma']<0.72) 
#     & (df['close/60ma']<0.77) 
#     & (df['close/60ma']>0.55) 
#     & (df['close/30ma']<0.82) 
# #     & (df['close/20ma']<0.82)
# #     & (df['close/5ma']<0.95)
#                 )
# print(strategy_mask2.sum())

9658

mean         0.056804
median       0.042696
max          0.839938
min         -0.735319
std          0.123248
size      9658.000000
Name: r, dtype: float64

mean         0.063536
median       0.032377
max          1.822414
min         -0.735319
std          0.142577
size      9658.000000
Name: r, dtype: float64

,mean,median,max,min,std,size
date,,,,,,
2011-03-31,0.199336,0.198529,0.207407,0.192071,0.007700,3
2011-06-30,NaN,NaN,NaN,NaN,NaN,0
2011-09-30,NaN,NaN,NaN,NaN,NaN,0
2011-12-31,-0.033480,-0.040872,0.243386,-0.157175,0.063833,257
2012-03-31,0.087218,0.054930,0.663144,-0.136259,0.110580,311
2012-06-30,NaN,NaN,NaN,NaN,NaN,0
2012-09-30,NaN,NaN,NaN,NaN,NaN,0
2012-12-31,0.084170,0.142196,0.191638,-0.104598,0.093225,56
2013-03-31,NaN,NaN,NaN,NaN,NaN,0


buy_at        max  max_idx   idx  is_selled  \
code      date                                                         
600892.SH 2020-04-24  18.969000  19.917450   2339.0  2338      False   
600734.SH 2020-04-27        NaN        NaN      NaN  1953      False   
          2020-04-24  11.708800  11.708800   1953.0  1952      False   
          2020-04-20  12.367420  12.733320   1949.0  1948       True   
600175.SH 2020-04-27        NaN        NaN      NaN  2318      False   
          2020-04-24  15.832320  16.002560   2318.0  2317      False   
600083.SH 2020-04-27        NaN        NaN      NaN  2369      False   
          2020-04-23   9.620000  10.310000   2368.0  2367       True   
          2020-04-22   8.660000  10.310000   2368.0  2366       True   
          2020-04-21   8.990000  10.310000   2368.0  2365       True   
          2020-04-20   9.370000   9.380000   2365.0  2364       True   
          2020-04-17   8.580000   9.390000   2364.0  2363       True   
          2020-04-16   7.790000   9.390000   2364.0  2362       True   
          2020-04-15   7.880000   9.390000   2364.0  2361       True   
          2020-04-14   8.120000   9.390000   2364.0  2360       True   
          2020-04-13   7.760000   9.390000   2364.0  2359       True   
          2020-04-10   7.930000   9.390000   2364.0  2358       True   
          2020-04-09   8.530000   8.570000   2358.0  2357       True   
          2020-04-08   8.450000   8.720000   2357.0  2356       True   
          2020-04-07   8.530000   8.720000   2357.0  2355       True   
600069.SH 2020-04-27        NaN        NaN      NaN  2375      False   
300362.SZ 2020-04-24   4.172780   4.533020   1144.0  1143      False   
002716.SZ 2020-04-27        NaN        NaN      NaN  1342      False   
          2020-04-21   5.533200   5.952960   1340.0  1338       True   
002711.SZ 2020-04-27        NaN        NaN      NaN  1398      False   
          2020-04-24   5.390250   5.893340   1398.0  1397      False   
002506.SZ 2020-04-27        NaN        NaN      NaN  1885      False   
          2020-04-24  13.354139  13.761901   1885.0  1884      False   
          2020-04-23  13.608991  13.863840   1884.0  1883      False   
          2020-04-22  14.067720  14.118690   1883.0  1882       True   
          2020-04-21  14.067720  14.118690   1882.0  1881       True   
          2020-04-20  14.118690  14.628389   1881.0  1880       True   
          2020-04-17  14.322570  14.628389   1881.0  1879       True   
          2020-04-16  14.628389  15.087121   1879.0  1878       True   
          2020-04-15  14.526450  15.087121   1879.0  1877       True   
          2020-04-14  15.138090  15.392941   1877.0  1876       True   
          2020-04-13  13.761901  15.392941   1877.0  1875       True   
          2020-04-10  14.322570  15.392941   1877.0  1874       True   
          2020-04-09  15.392941  15.392941   1874.0  1873       True   
          2020-04-08  15.494880  15.494880   1873.0  1872       True   
          2020-04-07  15.545850  15.596820   1872.0  1871       True   
002447.SZ 2020-04-27        NaN        NaN      NaN  2066      False   
002411.SZ 2020-04-27        NaN        NaN      NaN  2103      False   
          2020-04-24  33.619617  33.869579   2103.0  2102      False   
          2020-04-13  36.369179  38.868778   2095.0  2093       True   
          2020-04-10  35.994240  38.868778   2095.0  2092       True   
002176.SZ 2020-04-27        NaN        NaN      NaN  2361      False   
          2020-04-24  24.618299  24.747869   2361.0  2360      False   
000673.SZ 2020-04-27        NaN        NaN      NaN  2182      False   
          2020-04-08  33.143044  37.861320   2171.0  2169       True   

                        sell_at  sell_date         r  
code      date                                        
600892.SH 2020-04-24        NaN        NaT       NaN  
600734.SH 2020-04-27        NaN        NaT       NaN  
          2020-04-24        NaN        NaT       NaN  
     